<a href="https://colab.research.google.com/github/gacerioni/redis-workshop-json-search-vs/blob/master/gabs-short-intro-to-vector-db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop - Redis como VectorDB - QuickStart

## Vector Searches & Large Language Models

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)


Bem-vind[ao]s ao Workshop! Vamos ter uma experiência hands-on sobre alguns temas centrais do Redis, bem além do Caching.

Este material demonstra rapidamente como você pode usar o Redis como um VectorDB. Existe um hands-on mais completo, onde eu explico cada passo exatamente.

Quando falamos que existe um mundo de diferenças entre **SEARCH X QUERY**, a história começa aqui.

O Redis é o motor para buscas fonéticas, complexas, por sinonimo, por wildcard, por geolocalização, por ranges... e também por Vetor! Qualquer modelo que extraia a semântica por trás dos inputs é bem vindo aqui!

Espero que gostem!

# Objetivo do Laboratório

Lembre-se: Existe uma versão deste mesmo lab onde eu explico passo a passo o que estamos fazendo. Não se preocupe se você não entendeu de primeira. O objetivo aqui é de DEMO.

Basicamente, isso que faremos aqui:

*   Guardar uma base de dados de Bicicletas (como SKUs) no Redis.
*   Usar um modelo de Sentence Transformers da HuggingFace para gerar os embeddings a partir da descrição em linguagem natural da bicicleta.
*   Abrir um prompt de busca para o usuário final, onde iremos vetorizar a sua própria pergunta (input) usando o mesmo modelo de ML.
*   Efetuar uma busca de VSS/KNN contra o Redis, usando os embeddings (vectors) para encontrar a melhor resposta para a pergunta do usuário.



# Pré Requisito - Criar uma conta Free no Redis Cloud

Basta seguir o passo a passo [aqui](https://colab.research.google.com/github/gacerioni/redis-workshop-notebook-validator/blob/master/redis-workshop-setup-notebook-validator.ipynb)!

# Vector Similarity Search em 3 Passos!


## Passo 1 - Instalação e Configuração

In [ ]:
# Instale as dependências
!pip install -r https://raw.githubusercontent.com/gacerioni/redis-workshop-json-search-vs/master/deps/vector-intro/requirements.txt
!apt-get update
!apt-get install -y redis-tools

# Conecte ao Redis
import os
import redis

REDIS_HOST="redis-19343.c308.sa-east-1-1.ec2.redns.redis-cloud.com"
REDIS_PORT=19343
REDIS_PASSWORD="F7ouT6jN4LzIF0zOZkpABUo0AwTgHq8e"

redis_conn = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD,
    decode_responses=True
)

assert redis_conn.ping()

## Passo 2 - Carregar as Bikes e indexá-las no Redis

In [ ]:
import requests
from redis.exceptions import ResponseError
from redis.commands.search.field import TextField, NumericField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

# Carregar dados das bikes
URL = "https://raw.githubusercontent.com/gacerioni/redis-workshop-json-search-vs/master/deps/vector-intro/data/bikes.json"
bikes = requests.get(URL).json()
index_name = "idx:bikes_vss"

# Carregar dados no Redis
pipeline = redis_conn.pipeline()
for i, bike in enumerate(bikes, start=1):
    redis_key = f"bikes:{i:03}"
    pipeline.json().set(redis_key, "$", bike)
pipeline.execute()

# Função para checar se o index já existe
def index_exists(index_name):
    try:
        # Vai lançar um erro se o index não existir
        redis_conn.ft(index_name).info()
        return True
    except ResponseError:
        return False

# Usa a função para garantir uma operação atômica
if index_exists(index_name):
    print("Deleting older index version...")
    redis_conn.execute_command("FT.DROPINDEX", index_name)

# Criar índice para busca vetorial
VECTOR_DIMENSION = 384

schema = [
    TextField("$.model", as_name="model"),
    TextField("$.brand", as_name="brand"),
    NumericField("$.price", as_name="price"),
    TextField("$.description", as_name="description"),  # Adicionado o campo description
    VectorField("$.description_embeddings", "FLAT", {"TYPE": "FLOAT32", "DIM": VECTOR_DIMENSION, "DISTANCE_METRIC": "COSINE"}, as_name="vector")
]

index_def = IndexDefinition(prefix=["bikes:"], index_type=IndexType.JSON)
redis_conn.ft(index_name).create_index(fields=schema, definition=index_def)

## Passo 3 - Vetorização da descrição das Bikes e Prompt

Reta final! Vamos vetorizar as descrições das Bikes, guardar junto no JSON lá no Redis, e abrir o prompt pro usuário final.

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from redis.commands.search.query import Query
import pandas as pd

# Gerar embeddings
embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
keys = sorted(redis_conn.keys("bikes:*"))
descriptions = [redis_conn.json().get(key, "$.description")[0] for key in keys]
embeddings = embedder.encode(descriptions).astype(np.float32).tolist()

# Inserir embeddings no Redis
pipeline = redis_conn.pipeline()
for key, embedding in zip(keys, embeddings):
    pipeline.json().set(key, "$.description_embeddings", embedding)
pipeline.execute()

# Loop de busca interativa
while True:
    query = input("Digite sua pergunta: ")
    query_vec = embedder.encode([query]).astype(np.float32).tobytes()

    search_query = (
        Query("(*)=>[KNN 3 @vector $query_vector AS vector_score]")
        .sort_by("vector_score")
        .return_fields("vector_score", "model", "brand", "price", "description")
        .dialect(2)
    )

    results = redis_conn.ft("idx:bikes_vss").search(search_query, {"query_vector": query_vec}).docs

    data = []
    for doc in results:
        model = doc.__dict__.get('model', 'N/A')
        brand = doc.__dict__.get('brand', 'N/A')
        price = doc.__dict__.get('price', 'N/A')
        description = doc.__dict__.get('description', 'N/A')[:100]  # Limita a descrição a 100 caracteres
        vector_score = 1 - float(doc.__dict__.get('vector_score', 'N/A'))

        data.append({
            "Modelo": model,
            "Marca": brand,
            "Preço": price,
            "Score": f"{vector_score:.2f}",
            "Descrição": description
        })

    df = pd.DataFrame(data)
    print(df.to_markdown(index=False))